In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy
from scipy.spatial import distance
import os
from os import listdir
from os.path import isfile, join
import onnx
import onnxruntime as ort
import box_utils
import time

In [2]:
onnx_model = onnx.load('ultra_light_640.onnx')
ort_session = ort.InferenceSession('ultra_light_640.onnx')
input_name = ort_session.get_inputs()[0].name

In [3]:
onnx.__version__

'1.8.1'

In [4]:
ort.__version__

'1.7.0'

In [5]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense,GlobalAveragePooling2D,Lambda,add, Dropout, Activation,Concatenate,BatchNormalization
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras import backend as K

Using TensorFlow backend.


In [6]:
from inception_resnet_v1 import *
model = InceptionResNetV1()
model.load_weights('facenet_keras_weights.h5')

In [7]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [8]:
def preprocess(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [9]:
def predict(width, height, confidences, boxes, prob_threshold, iou_threshold=0.5, top_k=-1):
    boxes = boxes[0]
    confidences = confidences[0]
    picked_box_probs = []
    picked_labels = []
    for class_index in range(1, confidences.shape[1]):
        probs = confidences[:, class_index]
        mask = probs > prob_threshold
        probs = probs[mask]
        if probs.shape[0] == 0:
            continue
        subset_boxes = boxes[mask, :]
        box_probs = np.concatenate([subset_boxes, probs.reshape(-1, 1)], axis=1)
        box_probs = box_utils.hard_nms(box_probs,
                                       iou_threshold=iou_threshold,
                                       top_k=top_k,
                                       )
        picked_box_probs.append(box_probs)
        picked_labels.extend([class_index] * box_probs.shape[0])
    if not picked_box_probs:
        return np.array([]), np.array([]), np.array([])
    picked_box_probs = np.concatenate(picked_box_probs)
    picked_box_probs[:, 0] *= width
    picked_box_probs[:, 1] *= height
    picked_box_probs[:, 2] *= width
    picked_box_probs[:, 3] *= height
    return picked_box_probs[:, :4].astype(np.int32), np.array(picked_labels), picked_box_probs[:, 4]

In [10]:
import pickle
all_people_faces = pickle.load(open('all_people_faces.pkl','rb'))

In [11]:
def face_detect2(img2):
    raw_img = img2
    h, w, _ = raw_img.shape
    img = cv2.resize(raw_img, (640, 480))
    img_mean = np.array([127, 127, 127])
    img = (img - img_mean) / 128
    img = np.transpose(img, [2, 0, 1])
    img = np.expand_dims(img, axis=0)
    img = img.astype(np.float32)
    confidences, boxes = ort_session.run(None, {input_name: img})
    boxes, labels, probs = predict(w, h, confidences, boxes, 0.7)
    return boxes, labels, probs

In [12]:
def match_face(frame,margin =10):
    image = frame
    image1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes,labels,probs = face_detect2(image)
    for i in range(boxes.shape[0]):
        box = boxes[i, :]
        x1,y1,x2,y2 = box[0],box[1],box[2],box[3]
        if x1 <6:
            x1 =6
        elif y1<6:
            y1 =6
        face = image1[y1-margin//2:y2+margin//2,x1-margin//2:x2+margin//2,:]
        roi =  cv2.resize(face, (160,160))
        roi = np.expand_dims(roi, axis=0)
        captured_representation = l2_normalize(model.predict(preprocess(roi))[0])
        cv2.rectangle(image, (x1,y1), (x2,y2), (0,0,255), 1)
        cv2.rectangle(image,(x1,y1-20),(x2+40,y1),(0,255,0),-1)
        sim_dict = {}
        for i in all_people_faces:
            person_name = i
            representation = all_people_faces[i]
            similarity =  distance.euclidean(representation, captured_representation)
            sim_dict[person_name] = similarity
            min_similarity = min(sim_dict.values())
        found = 0
        if min_similarity < 0.8:
            person_name1 = [key for key in sim_dict if sim_dict[key] == min_similarity]
            cv2.putText(image, person_name1[0][:-1], (int(x1), int(y1-5)), 0, 0.6, (0, 0, 255), 2)
            found = 1
        if(found == 0):
            cv2.putText(image, 'unknown', (int(x1), int(y1-5)), 0, 0.6, (255, 0, 0), 2)
    if image.shape[1]> 400:
        im2 = cv2.resize(image,(int(image.shape[1]*0.6),int(image.shape[0]*0.6)))
    return im2

In [13]:
def face_recognition_vid(video_address):
    filename = video_address
    dir_name = os.path.splitext(filename)[0]
    parent_dir = "video_output/"
    directory = os.path.join(parent_dir, dir_name) 
    os.mkdir(directory)
    seconds = 1
    cap = cv2.VideoCapture(filename)

    # Get the frames per second
    fps = round(cap.get(cv2.CAP_PROP_FPS)) 
    multiplier = fps*seconds

    # Get the total numer of frames in the video.
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    print("FC :", frame_count)

    count = -1

    while cap.isOpened():
        ret, frame = cap.read()

        if ret:
            frame2 = match_face(frame)
            #cv2.imwrite('F:/Output/%s/Frame{:f}.jpg'.format(count),frame)
            cv2.imwrite('video_output/%s/image{:f}.jpg'.format(count)%(dir_name),frame2)
            count += multiplier 
            cap.set(1, count)
        else:
            cap.release()
            break

In [15]:
import tensorflow

In [17]:
face_recognition_vid('test_video1.mp4')

FC : 901.0


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node Conv2d_1a_3x3/convolution (defined at C:\Users\lalit\anaconda3\envs\base2\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_17656]

Function call stack:
keras_scratch_graph
